# Mini-Projet : débruitage d'image

*Viviane LESBRE*

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
from skimage.io import imread
img = imread("lena.png")
nx, ny = img.shape
npixels = nx*ny

## Question 1 :
On souhaite calculer la transformée de Fourier de l'image. Ensuite on implémente une fonction qui reconstruit l'image originale en ne conservant que les coefficients de Fourier correspondant aux $K$ fréquences les plus basses :


In [ ]:
def fourier(img):
    img_hat = np.fft.fft2(img)
    return (img_hat)

In [ ]:
def linear_approximation(img, K):
    img_hat = fourier(img)
    img_hat[K//2:-K//2, K//2:-K//2] = 0
    return (np.fft.ifft2(img_hat))

Pour calculer l'erreur $\Delta = \dfrac{\|I - I_r\|_2}{N}$ on fait une boucle sur les valeurs de K et on calcule pour chaque valeur le nouveau $\Delta$. POur calculer la norme on utilise la fonction de numpy `np.linalg.norm`

In [ ]:
I = np.fft.ifft(img)
Delta = []
for k in range(2,90):
    Ir = linear_approximation(img, k)
    dif = np.linalg.norm(I-Ir, 2)
    delta = dif/npixels
    Delta.append(delta )

plt.plot(Delta);

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(12, 24), sharex=True, sharey=True)
ax[0].imshow(img, cmap='gray')
ax[1].imshow(np.abs(img_approx_linear), cmap='gray')
for a in ax.ravel():
    a.set_axis_off()
plt.tight_layout()
plt.show()

## Question 2
On reconstruit l'image original à partir des $K$ coefficients de Fourier dont les amplitudes sont maximales.

In [ ]:
def nonlinear_approximation(img, ncoeffs):
    img_hat = np.fft.fft2(img)
    coeffs = np.sort(np.abs(img_hat).ravel())
    threshold = coeffs[-ncoeffs]
    mask = (np.abs(img_hat) >= threshold)
    return np.fft.ifft2(img_hat*mask).real

In [ ]:
K = 50000
print("Facteur de compression: " + str(npixels/K))
img_nonlineaire = nonlinear_approximation(img, K)
err = np.linalg.norm(img_nonlineaire - img)/npixels
print("Erreur de reconstruction: " + str(err))

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(12, 24), sharex=True, sharey=True)
ax[0].imshow(img, cmap='gray')
ax[1].imshow(img_nonlineaire, cmap='gray')
for a in ax.ravel():
    a.set_axis_off()
plt.tight_layout()
plt.show()

On remarque que l'essentiel de l'énergie de l'image se concentre sur un nombre restreint de coefficients, odnc même en en enlevant beaucoup de coefficients l'image est encore discernable.

## Question 3 :
On cherche à bruiter l'image par un bruit gaussien additif de moyenne nulle et d'écart-type $\sigma = 15$ :

In [ ]:
noise = np.random.normal(0, 15, size = (nx, ny))
img_bruitee = noise + img

fig, ax = plt.subplots(2, 2, figsize=(24, 24), sharex=True, sharey=True)
ax[0, 0].imshow(img, cmap='gray')
ax[0,0].text(15, 15, 'Photo originale', bbox={'facecolor': 'white', 'pad': 10})
ax[1, 0].imshow(img_bruitee, cmap='gray')
ax[1,0].text(15, 15, 'image bruitée', bbox={'facecolor': 'white', 'pad': 10})
ax[0, 1].imshow(noise, cmap='gray')
ax[0,1].text(15, 15, '''Bruit qu'on va ajouter''', bbox={'facecolor': 'white', 'pad': 10})
ax[1, 1].imshow(np.abs(np.fft.fft2(noise)), cmap='gray')
ax[1,1].text(15, 15, 'TTF du bruit', bbox={'facecolor': 'white', 'pad': 10})
for a in ax.ravel():
    a.set_axis_off()
plt.tight_layout()
plt.show()

On remarque bien une similitude du bruit dans l'espace réel et dans l'espace fréquentiel, cela est du à 

## Question 4
On débruite en fixant un seuil `threshold`:

In [ ]:
def fourier_thresholding(img, threshold):
    img_hat = np.fft.fft2(img)
    masque = (np.abs(img_hat) >= threshold)
    return np.fft.ifft2(img_hat*masque).real  #on ne s'interesse que à la partie réelle

On avait sans prétraiter la photo :

In [ ]:
err_orig = np.linalg.norm(img - img_bruitee)/npixels
print("L'erreur originale est de "  + str(err_orig))

Maintenant on fixe le seuil et on coupe toutes les valeurs supérieures

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(10, 20), sharex=True, sharey=True)

threshold = 25000

img_debruitee = fourier_thresholding(img_bruitee, threshold)
ax[0].imshow(img_bruitee, cmap='gray')
ax[1].imshow(img_debruitee, cmap='gray')
err = np.linalg.norm(img_debruitee - img)/npixels
print("Erreur pour le seuil " + str(threshold) + ": " + str(err))
    
for a in ax.ravel():
    a.set_axis_off()

## Question 5 :
On remplace la valeur de chaque pixel par la médiane des valeurs des pixels qui l'entoure, pour cela on utilise la bibliothèque `skimage`


In [ ]:
def median_filter(img, size):
    from skimage.filters import median
    from skimage.morphology import square
    img_median = median(img, selem=square(size))      
    return img_median

In [ ]:
size = 5
img_median = median_filter(img_bruitee, size)

err_median = np.linalg.norm(img - img_median)/npixels
print("L'erreur du filtre médian est : " + str(err_median))


fig, ax = plt.subplots(1, 2, figsize=(10, 20), sharex=True, sharey=True)
ax[0].imshow(img_bruitee, cmap='gray')
ax[0].set_title("Image bruitée")
ax[1].imshow(img_median, cmap='gray')
ax[1].set_title("Filtre médian appliqué")
for a in ax.ravel():
    a.set_axis_off()
    
plt.tight_layout()
plt.show()    

On utilise la médiane plutôt que la moyenne pour mieux prendre en compte les grands écarts de valeurs, donc augmenter le constraste.

## Question 6 :

On applique un filtrage par noyau gaussien à `img_bruitee` via la formule $G(x, y) = \dfrac{1}{2 \pi \sigma^2 }\exp \bigg (\dfrac{x^2 + y^2}{2 \sigma ^2} \bigg )$ et la bibliothèque scikit.

In [ ]:
from scipy.ndimage import gaussian_filter
img_g = gaussian_filter(img_bruitee, sigma=1.)

err_g = np.linalg.norm(img - img_g)/npixels
print("L'erreur du filtre Gaussien est: " + str(err_g))

fig, ax = plt.subplots(1, 2, figsize=(10, 20), sharex=True, sharey=True)
ax[0].imshow(img_bruitee, cmap='gray')
ax[0].set_title("Image bruitée")
ax[1].imshow(img_g, cmap='gray')
ax[1].set_title("Filtre gaussien appliqué")
for a in ax.ravel():
    a.set_axis_off()
    
plt.tight_layout()
plt.show()    

Ce filtrage a pour effet d'accentuer la netteté de l'image.

## Question 7a

On cherche à calculer le Laplacien de l'image bruitée, on a 
$$
\Delta T = \dfrac{\partial ^2 T}{\partial x^2} + \dfrac{\partial ^2 T}{\partial y^2}.
$$
On discrétise la dérivée seconde :
$$
\frac{\partial f²}{\partial² x} = \frac{f(x+dx)+f(x-dx)-2f(x)}{dx²}
$$

In [ ]:
def laplacian(img):
    nx, ny = img.shape
    #On symétrise les frontières
    img_f = np.zeros((nx + 2, ny + 2))
    img_f[1:-1, 1:-1] = img
    img_f[0, 1:-1] = img[0, :]
    img_f[-1, 1:-1] = img[-1, :]
    img_f[1:-1, 0] = img[:, 0]
    img_f[1:-1, -1] = img[:, -1]

    lapl_x = img_f[2:, :] - 2*img_f[1:-1, :] + img_f[:-2, :]
    lapl_y = img_f[:, 2:] - 2*img_f[:, 1:-1] + img_f[:,:-2]
    return lapl_x[:, 1:-1] + lapl_y[1:-1, :]

In [ ]:
img_lapl = laplacian(img)
fig, ax = plt.subplots(1, 2, figsize=(10, 20), sharex=True, sharey=True)
ax[0].imshow(img, cmap='gray')
ax[0].set_title("Photo originale")
ax[1].imshow(img_lapl, cmap='gray')
ax[1].set_title("Son laplacien")
for a in ax.ravel():
    a.set_axis_off()

## Question 7:
On résout l'équation de la diffusion à l'aide d'un schéma explicite en temps

In [ ]:
def diffusion(img, t, tstep):
    img = img.astype('float')
    s = 0
    while(s < t):
        img += tstep*laplacian(img)
        s += tstep
    return img

In [ ]:
t = 1.
tstep = 0.01
img_d = diffusion(img_bruitee, t, tstep)
err_d = np.linalg.norm(img - img_d)/npixels
print("L'erreur de diffusion est: " + str(err_d))

fig, ax = plt.subplots(1, 2, figsize=(10, 20), sharex=True, sharey=True)
ax[0].imshow(img_bruitee, cmap='gray')
ax[0].set_title("L'image bruitée")
ax[1].imshow(img_d, cmap='gray')
ax[1].set_title("Image diffusée")
for a in ax.ravel():
    a.set_axis_off()
    
plt.tight_layout()
plt.show()    